In [43]:
import numpy as np

import pandas as pd
import nltk 
from nltk.corpus import stopwords
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from sklearn.model_selection import train_test_split

In [101]:
df=pd.read_csv(r'C:\Users\hp\Desktop\amazon1.csv')

In [102]:
stopset =set(stopwords.words('english'))
vectorizer= TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=list(stopset))

In [103]:
df.head()

,Unnamed: 0,rating,reviewText,Unnamed: 3
0,0,1,This book was the very first bookmobile book I...,NaN
1,1,0,"When I read the description for this book, I c...",NaN
2,2,1,I just had to edit this review. This book is a...,NaN
3,3,1,I don't normally buy 'mystery' novels because ...,NaN
4,4,1,"This isn't the kind of book I normally read, a...",NaN


In [105]:
y=df.rating

In [106]:
x= vectorizer.fit_transform(df.reviewText)

In [107]:
print(x.shape)

(12000, 30323)


In [108]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2)

In [109]:
clf = naive_bayes.MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [110]:
roc_auc_score(y_test ,clf.predict_proba(x_test)[:,1])

0.9022792254286526

In [113]:
kindle_review_array=np.array(["IT IS NOT PRETTY BAD"])
kindle_review_vector= vectorizer.transform(kindle_review_array)
print(clf.predict(kindle_review_vector))

[0]
